# 1. SETTINGS

In [1]:
# libraries
import numpy as np
import pandas as pd
from datetime import date
import matplotlib.pyplot as plt
from IPython.display import display
import scipy.stats

In [2]:
# pandas options
pd.set_option("display.max_columns", None)

# 2. FUNCTIONS

In [3]:
### FUNCTION FOR COUNTING MISSINGS
def count_missings(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum() / data.isnull().count() * 100).sort_values(ascending = False)
    table = pd.concat([total, percent], axis = 1, keys = ["Total", "Percent"])
    table = table[table["Total"] > 0]
    return table

In [4]:
### FUNCTION FOR COMPUTING WEEK INDEX
def week_idx(date, end_date):
    # TODO: Hows does this work?
    return round((end_date - date).dt.days / 7 + 0.4)

# 3. DATA IMPORT

In [5]:
# import datasets
test  = pd.read_csv("../data/raw/Challenge_20180423.csv")
cust  = pd.read_csv("../data/raw/Customer.csv")
bond  = pd.read_csv("../data/raw/Isin.csv")
markt = pd.read_csv("../data/raw/Market.csv")
macro = pd.read_csv("../data/raw/MarketData_Macro.csv")
trade = pd.read_csv("../data/raw/Trade.csv")

In [6]:
# check all datasets
print("Test data:", test.shape)
display(test.head(3))
print("------------------------------")
print("Customer data:", cust.shape)
display(cust.head(3))
print("------------------------------")
print("Bonds data:", bond.shape)
display(bond.head(3))
print("------------------------------")
print("Market data:", markt.shape)
display(markt.head(3))
print("------------------------------")
print("Macroeconomic data:", macro.shape)
display(macro.head(3))
print("------------------------------")
print("Trade data:", trade.shape)
display(trade.head(3))

Test data: (484758, 6)


,PredictionIdx,DateKey,CustomerIdx,IsinIdx,BuySell,CustomerInterest
0,a1e0d80784,20180423,1856,13323,Buy,NaN
1,c2cc6cc2a8,20180423,1856,9230,Buy,NaN
2,a8e94f6344,20180423,1780,9157,Buy,NaN


------------------------------
Customer data: (3471, 5)


,CustomerIdx,Sector,Subsector,Region,Country
0,2975,Banks and Intermediaries,Bank,Americas,BARBADOS
1,1594,Asset Managers & Hedge Funds,NaN,Americas,BERMUDA
2,399,Corporation,Corp - Comm. & Prof. Services,Americas,BERMUDA


------------------------------
Bonds data: (27411, 17)


...

,IsinIdx,DateKey,Price,Yield,ZSpread
0,1,20160101,104.250,7.835,5.505
1,7,20160101,107.500,7.520,5.541
2,102,20160101,100.746,4.048,2.085


------------------------------
Macroeconomic data: (877, 112)


,DateKey,SSE,DAX,EUROSTOXX,VSTOXX,FTSE100,HSI,NIKKEI,DOWJONES_INDU,SP500,VIX,FX_USD.ARS,FX_USD.AUD,FX_USD.BRL,FX_USD.CAD,FX_USD.CHF,FX_USD.CNO,FX_USD.CNY,FX_USD.EUR,FX_USD.GBP,FX_USD.HKD,FX_USD.IDR,FX_USD.JPY,FX_USD.NOK,FX_USD.SGD,FX_USD.TRY,FX_USD.ZAR,MoneyMarket_ARS3M,MoneyMarket_AUD3M,MoneyMarket_CAD3M,MoneyMarket_CHF3M,MoneyMarket_CNO3M,MoneyMarket_CNY3M,MoneyMarket_EUR3M,MoneyMarket_GBP3M,MoneyMarket_HKD3M,MoneyMarket_IDR3M,MoneyMarket_JPY3M,MoneyMarket_NOK3M,MoneyMarket_SGD3M,MoneyMarket_TRY3M,MoneyMarket_USD3M,MoneyMarket_ZAR3M,Swap_ARS10Y,Swap_ARS2Y,Swap_ARS5Y,Swap_AUD10Y,Swap_AUD2Y,Swap_AUD30Y,Swap_AUD5Y,Swap_BRL10Y,Swap_BRL2Y,Swap_BRL5Y,Swap_CAD10Y,Swap_CAD2Y,Swap_CAD30Y,Swap_CAD5Y,Swap_CHF10Y,Swap_CHF2Y,Swap_CHF30Y,Swap_CHF5Y,Swap_CNH10Y,Swap_CNH2Y,Swap_CNH30Y,Swap_CNH5Y,Swap_CNO10Y,Swap_CNO2Y,Swap_CNO30Y,Swap_CNO5Y,Swap_CNY10Y,Swap_CNY2Y,Swap_CNY30Y,Swap_CNY5Y,Swap_EUR10Y,Swap_EUR2Y,Swap_EUR30Y,Swap_EUR5Y,Swap_GBP10Y,Swap_GBP2Y,Swap_GBP30Y,Swap_GBP5Y,Swap_HKD10Y,Swap_HKD2Y,Swap_HKD30Y,Swap_HKD5Y,Swap_IDR10Y,Swap_IDR2Y,Swap_IDR30Y,Swap_IDR5Y,Swap_JPY10Y,Swap_JPY2Y,Swap_JPY30Y,Swap_JPY5Y,Swap_NOK10Y,Swap_NOK2Y,Swap_NOK30Y,Swap_NOK5Y,Swap_SGD10Y,Swap_SGD2Y,Swap_SGD30Y,Swap_SGD5Y,Swap_TRY10Y,Swap_TRY2Y,Swap_TRY5Y,Swap_USD10Y,Swap_USD2Y,Swap_USD30Y,Swap_USD5Y,Swap_ZAR10Y,Swap_ZAR2Y,Swap_ZAR30Y,Swap_ZAR5Y
0,20150101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.20,0.116959,0.81620,0.377658,0.861030,1.006264,0.161512,0.163399,1.2101,1.55775,0.128941,0.000081,0.008341,0.133451,0.756287,0.428964,0.086457,35.230452,2.724540,1.303309,-0.063143,3.701588,3.617925,0.077487,0.561227,0.381829,7.17183,0.112078,1.459648,0.709986,10.001685,0.256570,6.130798,19.989643,29.980614,21.985733,3.041150,2.398950,3.625076,2.558248,NaN,NaN,NaN,2.277279,NaN,2.760043,1.775859,0.409878,NaN,1.082563,-0.046046,3.749868,3.418686,NaN,3.549519,3.750081,3.419989,NaN,3.550031,3.439794,3.218679,NaN,3.239508,0.697389,NaN,1.408397,0.233572,NaN,NaN,NaN,NaN,2.252274,0.95467,NaN,1.814259,8.955845,8.368177,9.552123,8.708471,0.444559,NaN,1.233032,0.180984,1.806932,NaN,2.179804,1.295793,NaN,NaN,NaN,NaN,9.885583,10.069276,9.915120,2.295037,NaN,2.712112,1.774452,7.963276,6.856526,8.331874,7.403867
1,20150102,NaN,9764.73,3139.44,26.2531,6547.80,23857.82,NaN,17832.99,2058.20,17.79,0.116891,0.81140,0.371292,0.851136,0.998851,0.161186,0.163292,1.2008,1.53340,0.128934,0.000080,0.008308,0.131841,0.752757,0.427241,0.085609,33.000000,2.736281,1.302850,-0.063000,3.701588,3.617925,0.077179,0.563380,0.385000,7.10000,0.112078,1.490000,0.790000,10.020000,0.255218,6.125000,20.000000,30.000000,22.000000,3.108394,2.452905,3.692179,2.625847,11.767860,12.807338,12.202677,2.231223,1.445105,2.716551,1.748069,0.367391,NaN,1.040147,-0.058452,3.750081,3.419989,NaN,3.550031,3.750081,3.419989,NaN,3.550031,3.439794,3.218679,NaN,3.239508,0.640191,0.062120,1.347191,0.211268,1.674890,0.810358,2.109754,1.305976,2.290000,0.97500,2.75196,1.840000,8.950000,8.350000,9.550000,8.700000,0.444559,NaN,1.233032,0.180984,1.794443,1.100220,2.168744,1.280511,2.447394,1.132505,2.95190,1.948474,9.946260,10.151881,9.976101,2.236910,0.893242,2.647499,1.746344,7.940000,6.840000,8.310000,7.390000
2,20150105,3350.519,9473.16,3023.14,29.6236,6417.16,23721.32,17408.71,17501.65,2020.58,19.92,0.116782,0.80905,0.366757,0.850268,0.993764,0.160822,0.163265,1.1939,1.52555,0.128926,0.000079,0.008366,0.131277,0.749260,0.429304,0.085426,32.000000,2.742523,1.303171,-0.063000,3.749732,3.691439,0.075252,0.563380,0.383570,7.10000,0.112140,1.450000,0.875000,9.993400,0.254194,6.125000,20.000000,30.000000,22.000000,3.007227,2.423082,3.591836,2.566016,11.809585,12.736379,12.154331,2.180266,1.420899,2.668708,1.713829,0.350011,-0.263049,0.981356,-0.069792,3.700010,3.405070,NaN,3.500016,3.700010,3.405070,NaN,3.500016,3.450010,3.230070,NaN,3.250016,0.658914,0.063256,1.351414,0.220293,1.613752,0.799833,2.043961,1.251222,2.190000,0.94000,2.65280,1.750000,8.950000,8.350000,9.550000,8.700000,0.436972,0.112315,1.225011,0.178599,1.747772,1.063624,2

------------------------------
Trade data: (6762021, 8)


,TradeDateKey,CustomerIdx,IsinIdx,BuySell,NotionalEUR,Price,TradeStatus,CustomerInterest
0,20161207,2789,8478,Sell,653168.0,0.0,Unknown,1.0
1,20170329,2574,14562,Buy,1656487.0,0.0,Unknown,1.0
2,20170418,2574,4747,Buy,939673.0,0.0,Unknown,1.0


# 4. PREPROCESSING

## 4.1. TRADE & TEST DATA

#### SOME CHECKS

In [7]:
# check missings
count_missings(trade)

,Total,Percent
Price,4617933,68.292201


In [8]:
# descriptive stats
trade.describe()

,TradeDateKey,CustomerIdx,IsinIdx,NotionalEUR,Price,CustomerInterest
count,6.762021e+06,6.762021e+06,6.762021e+06,6.762021e+06,2.144088e+06,6.762021e+06
mean,2.016750e+07,1.922354e+03,1.460369e+04,6.313228e+06,1.779008e+05,3.271106e-01
std,6.768082e+03,8.579293e+02,7.963809e+03,2.712408e+08,1.318503e+06,4.691581e-01
min,2.016010e+07,0.000000e+00,0.000000e+00,-1.485547e+08,-9.999990e+05,0.000000e+00
25%,2.016063e+07,1.288000e+03,7.392000e+03,3.410980e+05,9.313780e+01,0.000000e+00
50%,2.017023e+07,2.090000e+03,1.522900e+04,8.812120e+05,1.010000e+02,0.000000e+00
75%,2.017093e+07,2.574000e+03,2.211900e+04,2.136842e+06,1.075000e+02,1.000000e+00
max,2.018042e+07,3.470000e+03,2.739400e+04,2.000000e+11,9.999999e+06,1.000000e+00


The number of observations and bonds per customer is different in trade and test data. Trade data only contains bonds that a given customer has actually traded. In test data, for each customer, the set of bonds is only a subset of the bonds that he actually traded in the past (but not the whole set, which leads to a smaller number of observations per customer in the test data). Also, the test set is biased towards very active traders.

In [9]:
# check if bonds in test are a subset of bonds in trade
# print "No" if bonds in test are new for that customer
for i in test.CustomerIdx.unique():
    A = trade[trade.CustomerIdx == i].IsinIdx.unique()
    B = test[test.CustomerIdx == i].IsinIdx.unique()
    C = set(B).issubset(set(A))
    if C == False: 
        print("No!")
print("Finished")

Finished


In [10]:
# check number of bonds per customer
display(trade.groupby("CustomerIdx").IsinIdx.nunique().describe())
print("------------------------------")
display(test.groupby("CustomerIdx").IsinIdx.nunique().describe())

count     3439.000000
mean       405.832800
std       1145.875089
min          1.000000
25%          5.000000
50%         33.000000
75%        233.500000
max      14132.000000
Name: IsinIdx, dtype: float64

------------------------------


count    2495.000000
mean       97.145892
std       301.375388
min         1.000000
25%         3.000000
50%        12.000000
75%        58.000000
max      5044.000000
Name: IsinIdx, dtype: float64

#### PREPROCESSING

Target equals 0 if TradeStatus = "Holding" and 1 in all other cases. The holding operations are artificial and do not actually mean a customer-bond interaction on the considered market, so we drop these observations from the dataset.

In [11]:
# create target variable
trade["CustomerInterest"] = 1
trade["CustomerInterest"][trade["TradeStatus"] == "Holding"] = 0
trade.CustomerInterest.mean()

/Users/hauptjoh/anaconda/envs/deeplearning/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


0.32711063748545

In [12]:
# deleting holding cases
trade = trade[trade["TradeStatus"] != "Holding"]

Test data does not contain features "Price", "NotionalEUR", so it is not possible to use them as predictors. 

In [13]:
# delete features
del trade["NotionalEUR"]
del trade["Price"]

In [14]:
# convert dates
trade["TradeDateKey"] = pd.to_datetime(trade["TradeDateKey"], format = '%Y%m%d')

In [15]:
# add week index
trade["Week"] = week_idx(trade["TradeDateKey"], pd.Timestamp('2018-04-23 00:00:00'))
trade["Week"] = trade["Week"].max() + 1 - trade["Week"]
test["Week"]  = trade["Week"].max() + 1

In the test data, each observation covers one week, whereas the training data is on a daily basis. We can aggregate the training data to a week level to have the same granularity. The target variable is computed as max over a week, whereas for different features we can compute different stats describing behavior during that week (e.g. mean, sd, range, etc).

In [16]:
# aggregate weekly data: target = 1 if there is at least single 1 during week
trade = trade.groupby(["CustomerIdx", "Week", "IsinIdx", "BuySell"], as_index = False).CustomerInterest.max()

The training data contains 0 only if a customer has explicitly stated that she holds a bond. However, there are a lot of 0 missing from the table for the cases when a customer does not interact with a bond at all. Hence, it is necessary to impute these missing observations. For each customer, we can look at the set of bonds that he ever traded and add all missing weeks for each of that bonds as new rows with target = 0 (the rows when customer did not interact with a specific bond). That significantly increases the sample size but also makes data much closer to the test set.

Current implementation increases the sample size from 1.7m to 111m rows (~3Gb). This takes a few minutes.

In [ ]:
# add missing weeks
print(trade.shape)
trade = trade.groupby(["CustomerIdx", "Week", "IsinIdx", "BuySell"]).CustomerInterest.unique().unstack("Week").stack("Week", dropna = False)
trade = trade.reset_index()
trade.columns = ["CustomerIdx", "IsinIdx", "BuySell", "Week", "CustomerInterest"]
print(trade.shape)

Mean "CustomerInterest" should be around 1.5% according to the organizers.

In [17]:
# fill new cases with 0 in target
trade.CustomerInterest.fillna(0, inplace = True)
trade["CustomerInterest"] = trade.CustomerInterest.astype(int)
trade.CustomerInterest.mean()

1.0

## 4.2. CUSTOMER DATA

There are 3471 unique customers in the training data of which 2495 show up in the test set. There are no unknown customers in the test set.

In [18]:
# unique customers
cust.CustomerIdx.nunique()

3471

In [19]:
# Compare number of test customers showing up in cust set to number of unique customers in test set
np.sum(np.in1d(test.CustomerIdx.unique(), cust.CustomerIdx.unique()))/test.CustomerIdx.nunique()

1.0

In [20]:
trade.columns

Index(['CustomerIdx', 'Week', 'IsinIdx', 'BuySell', 'CustomerInterest'], dtype='object')

More than 20% of customers are interested only once in the training period, more than 50% are interested less than 15 times total -> There are a few customers that are interested a lot, maybe we should focus on them

In [21]:
# check percentiles
np.percentile(trade.loc[trade.CustomerInterest==1].groupby("CustomerIdx").size(), range(0,100,10))

array([   1. ,    1. ,    3.6,    7. ,   14. ,   27. ,   57. ,  127. ,
        295.4,  864.2])

Customer information:
- 5 different sectors
- 41 subsectors
- 3 regions
- 99 countries

We can include sectors and regions directly, but may want to reduce the dimension of subsectors and countries

In [22]:
# check columns
cust.columns

Index(['CustomerIdx', 'Sector', 'Subsector', 'Region', 'Country'], dtype='object')

In [23]:
# create dummies for customers
cust_dummies = pd.concat([cust.CustomerIdx, pd.get_dummies(cust.Sector), pd.get_dummies(cust.Region)], axis = 1)

Most customers are asset managers or banks/intermediaries (possibly high volume groups?). Distribution over Americas, Europe/Africa, Asia is 53/25/22

In [24]:
# merge customer dummies
trade = trade.merge(cust_dummies, on = "CustomerIdx", how = "left")
test  = test.merge(cust_dummies,  on = "CustomerIdx", how = "left")

Asset Managers/Hedge funds and Banks/Intermediaries show clearly more interest than privat and corporate investors

In [25]:
# distribution of customer types
trade.loc[trade.CustomerInterest==1,['Asset Managers & Hedge Funds', 'Asset Owners',
       'Banks and Intermediaries', 'Corporation', 'Official Institution - OI']].mean()

Asset Managers & Hedge Funds    0.673504
Asset Owners                    0.035005
Banks and Intermediaries        0.269887
Corporation                     0.002241
Official Institution - OI       0.019363
dtype: float64

## 4.4. BONDS DATA

Check that bonds after maturity are not tradeable. If this is correct, the easiest solution would be to manually correct these to 0 in model predictions

In [26]:
# descriptive stats
bond.describe(include = "all")

,IsinIdx,TickerIdx,ActualMaturityDateKey,IssueDateKey,Seniority,Currency,ActivityGroup,Region,Activity,RiskCaptain,Owner,CompositeRating,IndustrySector,IndustrySubgroup,MarketIssue,IssuedAmount,CouponType
count,27411.000000,27411.000000,2.741100e+04,2.741100e+04,27411,27411,27411,27411,27411,27411,27411,27411,27406,27406,27394,2.741100e+04,27411
unique,NaN,NaN,NaN,NaN,9,23,3,8,16,38,105,29,13,338,14,NaN,6
top,NaN,NaN,NaN,NaN,SEN,USD,FLOW G10,AMERICAS,HG CASH,HG CASH NONFIN,US NONFIN 5Y AND IN CASH,NR,Financial,Commer Banks Non-US,Euro mtn,NaN,FIXED
freq,NaN,NaN,NaN,NaN,21233,16358,16552,10732,7928,5608,1714,4874,10046,3715,7374,NaN,24105
mean,13705.000000,1748.038598,2.023912e+07,2.013163e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.044049e+09,NaN
std,7913.018451,1028.719783,8.515510e+04,4.165186e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.274043e+10,NaN
min,0.000000,0.000000,2.015043e+07,1.955010e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN
25%,6852.500000,855.000000,2.019042e+07,2.012013e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.000000e+08,NaN
50%,13705.000000,1753.000000,2.022013e+07,2.014062e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.500000e+08,NaN
75%,20557.500000,2647.000000,2.025091e+07,2.016071e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.200000e+09,NaN


Create bond dummies for model training and merging into trade data

In [27]:
# create bond dumiies
bond_dummies = pd.concat([bond.IsinIdx,
                          bond.Currency, # Used later to merge currency data
                          pd.get_dummies(bond.ActivityGroup), 
                          pd.get_dummies(bond.CompositeRating)], axis = 1)

In [28]:
# merge bond dummies
trade = trade.merge(bond_dummies, on = "IsinIdx", how = "left")
test  = test.merge(bond_dummies,  on = "IsinIdx", how = "left")

## MACROECONOMIC DATA

We have >100 variables here:
- Stock indices (DAX, FTSE100, ...)
- Volatility indices (VSTOXX, VIX, )
- Currency exchange rates (USD <> EUR/CNY/...)
- Inter-bank money lending rate (Money Market) 3-month for each currency 
- Mid- to long-term swaps (2-10 years). TODO: Unsure of the effect on bond trades

In [29]:
# descriptive stats
macro.columns.tolist()[0:15]

['DateKey',
 'SSE',
 'DAX',
 'EUROSTOXX',
 'VSTOXX',
 'FTSE100',
 'HSI',
 'NIKKEI',
 'DOWJONES_INDU',
 'SP500',
 'VIX',
 'FX_USD.ARS',
 'FX_USD.AUD',
 'FX_USD.BRL',
 'FX_USD.CAD']

Heuristically fill missing values with the previous value or 2xprevious value. If still missing, fill values with the following or 2xfollowing value

In [30]:
macro = macro.fillna(macro.shift(1)).fillna(macro.shift(2)).fillna(macro.shift(-1)).fillna(macro.shift(-2))

In [31]:
# convert dates
macro["DateKey"] = pd.to_datetime(macro["DateKey"], format = '%Y%m%d')

In [32]:
# add week index
macro["Week"] = week_idx(macro["DateKey"], pd.Timestamp('2018-04-23 00:00:00'))
macro["Week"] = macro["Week"].max() + 1 - macro["Week"]

Aggregate the macro values by week. 

TODO: We could also take the lag first and then aggregate, not sure what makes more sense (JH).

In [33]:
macro = macro.groupby(["Week"]).agg("mean")

We are interested in the change in the macro variable compared to the previous date, I think, to check if e.g. the currency value went up or down

In [34]:
# Replace missing lag for first week with 0
macro_diff1 = (macro - macro.shift(1)).fillna(0)

TODO: I think it makes sense to create a common variable e.g. "currency trend" that relates to the specific currency of the bond and/or holder.

In [35]:
fx_diff1 = macro_diff1.filter(like="FX",axis=1)

In [36]:
fx_diff1["USD"] = 1

/Users/hauptjoh/anaconda/envs/deeplearning/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [37]:
fx_diff1 = fx_diff1.reset_index().melt(id_vars="Week", var_name="Currency", value_name="Currency_trend1w")

In [38]:
fx_diff1.Currency = fx_diff1.Currency.str[-3:]

The currency data 'fx' can be merged into the bond data

In [39]:
fx_diff1.Currency.unique()

array(['ARS', 'AUD', 'BRL', 'CAD', 'CHF', 'CNO', 'CNY', 'EUR', 'GBP',
       'HKD', 'IDR', 'JPY', 'NOK', 'SGD', 'TRY', 'ZAR', 'USD'], dtype=object)

In [40]:
trade = trade.merge(fx_diff1, how='left', on=["Week","Currency"])
test = test.merge(fx_diff1, how='left', on=["Week","Currency"])

There are a few weird currencies (or typos?) for which we don't have information, e.g. CNH

In [41]:
trade.Currency_trend1w.fillna(1, inplace=True)
test.Currency_trend1w.fillna(1, inplace=True)

## 4.5. MARKET DATA

# 5. DATA EXPORT

In [42]:
# check dimensions
print(trade.shape)
print(test.shape)

(1772720, 47)
(484758, 49)


In [44]:
count_missings(trade)

,Total,Percent


In [45]:
count_missings(test)

,Total,Percent
CustomerInterest,484758,100.0


In [ ]:
# export CSV
trade.to_csv("../data/prepared/train_new.csv", index = False, float_format = "%.4f")
test.to_csv("../data/prepared/test_new.csv",   index = False, float_format = "%.4f")